In [38]:
import warnings 
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from plotnine import *

columns = ["name","backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
#lgb1 = pd.read_pickle("lgbInvtSim_L1.pkl")#[columns] #_L_
#ets1 = pd.read_pickle("etsInvtSim_L1.pkl")#[columns] #_L_
#Idf1      = pd.concat([lgb1,ets1])

from tqdm import tqdm
lp = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}

'''
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10" :((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9":((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}'''
'''
def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]*gap:lp[i][1]*gap])
        l2 = np.array(df['true_demand'][lp[i][0]*gap:lp[i][1]*gap])
        if back_:
            asl2 = []
            for j in range(len(l1)):
                if l1[j] == 0 or l2[j] == 0:
                    a = 1 #if  else 
                else:
                    if (l1[j]/l2[j]) <= 1:
                        a = 1-(l1[j]/l2[j])
                    else:
                        a = 0
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl
'''
def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]*gap:lp[i][1]*gap])
        l2 = np.array(df['true_demand'][lp[i][0]*gap:lp[i][1]*gap])
        if back_:
            asl2 = []
            for j in range(len(l1)):
                if l1[j] == 0:
                    a = 1 #if  else 
                else:
                    a = 0
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl
def levels_summary2(dff:pd.DataFrame, 
                   col_1:str, 
                   col_2:str,
                   col_3:str,
                   lp_:dict, 
                   name_:str,
                   tsl_: float,
                   senario_:str,
                   c = True , 
                   gap_:int = 28, 
                   lvls:int = 12, lvls_list = [1,2,4,3,5,6,7,8,10,9,11,12]):
    dff_                           = pd.DataFrame()
    dff_['Levels']                 = lvls_list
    dff_['Forecast_Methods']       = [name_]*lvls
    dff_['Target_Service_Level']   = [tsl_]*lvls
    dff_['Senario']                = [senario_]*lvls
    dff_['Achieved_Service_Level'] = check(lp = lp_, df = dff, col = col_1, back_ = c, gap = gap_)
    dff_['Holding_Costs']          = check(lp = lp_, df = dff, col = col_2, back_ = not c, gap = gap_)
    dff_['Backlogging_Costs']      = check(lp = lp_, df = dff, col = col_3, back_ = not c, gap = gap_)
    return dff_
def asl_chcb(lp:dict, Idf:pd.DataFrame, names, names_m, col_list3, senario:str, lvls_ = 3, lvls_list_ = [10,11,12]):
    summary_list = []
    for i in tqdm(range(len(names))):
        idf = Idf[Idf['name'] == names[i]]
        for j in col_list3:
            summary_list.append(levels_summary2(lp_ = lp, 
                                            dff= idf, 
                                            name_ = names_m[i],
                                            col_1 = j[0], 
                                            col_2 = j[1], 
                                            col_3 = j[2], 
                                            tsl_ = j[3], senario_ = senario, lvls = lvls_, lvls_list = lvls_list_))
    return pd.concat(summary_list)

In [8]:
lvls = []
for i in lp.keys():
    a,b = lp[i]
    lvls.append([i[1:]]*28*(b-a))
   # n+=1
levels = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)

In [ ]:
sst = ['true_demand','forecasts','sst_90','backlog_90','sst_95','backlog_95','sst_99','backlog_99']
col = ['levels','true_demand','forecasts']
lgb1 = pd.read_pickle(f"721lgbInvtSim_L{1}.pkl")#[columns]
a = pd.concat([levels, lgb1[lgb1['name']=='lgb_base']],axis=1)
a[sst].iloc[:28,:]#[(a['levels'] == "10")&(a['backlog_90']>0)]#&(a['backlog_90']>0)

In [4]:
f = a['forecasts']
t = a['true_demand']
(f-t).var()

9.691357288578738

## COSTS & ACHIEVED SERVICE LEVEL

## Inventory performance 1:

In [ ]:
col_list1 = ['backlog_90','backlog_95','backlog_99']
col_list2 = ['ch_90','cb_90','ch_95','cb_95','ch_99','cb_99']
LGBM      = levels_summary(lp_ = lp, dff= lgbbase3, col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM')
LGBM_BU   = levels_summary(lp_ = lp, dff= lgbbu3,   col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_BU')
LGBM_TD   = levels_summary(lp_ = lp, dff= lgbtd3,   col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_TD')
LGBM_MinT = levels_summary(lp_ = lp, dff= lgbmint3, col_1 = col_list1, col_2 = col_list2, name_ = 'LGBM_MinT')

ETS       = levels_summary(lp_ = lp, dff= etsbase3, col_1 = col_list1, col_2 = col_list2, name_ = 'ETS')
ETS_BU    = levels_summary(lp_ = lp, dff= etsbu3,   col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_BU')
ETS_TD    = levels_summary(lp_ = lp, dff= etstd3,   col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_TD')
ETS_MinT  = levels_summary(lp_ = lp, dff= etsmint3, col_1 = col_list1, col_2 = col_list2, name_ = 'ETS_MinT')

base_invt = pd.concat([LGBM,LGBM_BU,LGBM_TD,LGBM_MinT,ETS,ETS_BU,ETS_TD,ETS_MinT])
#base_invt.to_pickle("base_ls_1.pkl")

In [ ]:
l1 = base_invt[base_invt.index == 0]
l1

## 2： Gnerated Performance for Visualization

### test:

In [21]:
method = "wls-var"
MinT = pd.read_pickle(f"TDFPOrder_L1_10-12.pkl")[['ot_90','ot_95','ot_99']]#[columns]#_L1
MinT

,ot_90,ot_95,ot_99
0,7.338106,8.939324,11.942942
1,2.989336,2.989336,2.989336
2,5.119798,5.119798,5.119798
3,8.181289,8.181289,8.181289
4,0.068346,0.068346,0.068346
...,...,...,...
1195203,0.002681,0.002681,0.002681
1195204,0.000795,0.000795,0.000795
1195205,0.000000,0.000000,0.000000
1195206,0.000000,0.000000,0.000000


In [23]:
i=1
lgb1 = pd.read_pickle(f"lgbInvtSim_L{i}.pkl")#[columns] #_L_
ets1 = pd.read_pickle(f"etsInvtSim_L{i}.pkl")#[columns] #_L_zhme
BASE = pd.concat([lgb1,ets1])[['ot_90','ot_95','ot_99']]
BASE

,ot_90,ot_95,ot_99
0,0.000000,0.000000,131.192218
1,35890.603430,36044.873794,36203.067121
2,34107.868750,34107.868750,34107.868750
3,34764.195891,34764.195891,34764.195891
4,39444.813836,39444.813836,39444.813836
...,...,...,...
1199515,0.000000,0.000000,0.000000
1199516,0.001689,0.001689,0.001689
1199517,0.030038,0.030038,0.030038
1199518,0.029265,0.029265,0.029265


In [21]:
columns = ["name","true_demand","backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
method = "wls-var"
MinT = pd.read_pickle(f"MinTOrder_L3_{method}.pkl")[columns]#_L1
col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT','ETS','ETS_BU','ETS_TD','ETS_MinT']
lp2 = {"l10":(0, 3049), "l11":(3049, 3049+4), "l12":( 3049*4, 3049*(14))}
L_BU  = asl_chcb(lp=lp2, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'MinT',lvls_= 3, lvls_list_ = [10,11,12])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.04it/s]


###  Results:

In [29]:

def collect(i: int, method):
    lp = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}
    
    columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
    col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
    names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT','ETS','ETS_BU','ETS_TD','ETS_MinT']
    lgb1 = pd.read_pickle(f"721lgbInvtSim_L{i}.pkl")[columns] #_L_
    ets1 = pd.read_pickle(f"721etsInvtSim_L{i}.pkl")[columns] #_L_zhme
    BASE = pd.concat([lgb1,ets1])
    BU   = pd.read_pickle(f"721BUOrder_L{i}.pkl")[columns]
    TDFP = pd.read_pickle(f"721TDFPOrder_L{i}.pkl")[columns]
    MinT = pd.read_pickle(f"721MinTOrder_{method}_L{i}.pkl")[columns]#_L1
    #lp2_ = {"l10":(0, 3049), "l11":(3049, 3049+4), "l12":( 3049*4, 3049*(14))}
    names     = ['lgb_base',  'ets_base']
    names_m   = ['LGBM','ETS']
    L_BASE  = asl_chcb(lp=lp, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BASE')
    
    names     = ['lgb_base', 'lgb_bu',  'ets_base', 'ets_bu']
    names_m   = ['LGBM','LGBM_BU','ETS','ETS_BU']
    L_BU    = asl_chcb(lp=lp, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BU_IR')
    
    names   = ['lgb_base','lgb_td', 'ets_base','ets_td']
    names_m = ['LGBM','LGBM_TD','ETS','ETS_TD']
    L_TDFP  = asl_chcb(lp=lp, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TD_IR', lvls_ = 3, lvls_list_ = [10, 11, 12])#, lvls_ = 3, lvls_list_ = [10, 11, 12]
    
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_MinT  = asl_chcb(lp=lp, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'MinT_IR', lvls_ = 3, lvls_list_ = [10, 11, 12])
    all_ = pd.concat([L_BASE,L_BU,L_TDFP,L_MinT])
    return all_#mint_ols
for lead_time in [1]:
    print(f"Lead_time is {lead_time}")
    method_ = "wls-var"
    all_ = collect(i= lead_time, method = method_)
    all_.to_csv(f"721Base2IR_L{lead_time}_{method_}.csv", index = False)

Lead_time is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.68s/it]


In [22]:
B = all_[all_['Senario'] == 'TD_IR']
B[B['Levels'] == 11]

,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs
1,11,LGBM,90%,TD_IR,0.767512,2.545948,4.664029
1,11,LGBM,95%,TD_IR,0.808501,3.023031,7.765685
1,11,LGBM,99%,TD_IR,0.869009,3.979858,26.136685
1,11,LGBM_TD,90%,TD_IR,0.770050,2.530156,4.521904
1,11,LGBM_TD,95%,TD_IR,0.811222,3.009623,7.510930
1,11,LGBM_TD,99%,TD_IR,0.871605,3.970445,25.204815
1,11,ETS,90%,TD_IR,0.783465,2.697654,4.225148
1,11,ETS,95%,TD_IR,0.824943,3.235274,6.956361
1,11,ETS,99%,TD_IR,0.883213,4.303771,22.916732
1,11,ETS_TD,90%,TD_IR,0.783879,2.687708,4.135635


### MinT:

In [39]:
def collect(i: int, method = ['shrink', 'ols', 'wls-structure',"wls-var"]):
    columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
    col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu','ets_td', 'ets_mint']
    names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT','ETS','ETS_BU','ETS_TD','ETS_MinT']
    #lgb1 = pd.read_pickle(f"721lgbInvtSim_L{i}.pkl")[columns] #_L_
    #ets1 = pd.read_pickle(f"etsInvtSim_L{i}.pkl")[columns] #_L_zhme
    #BASE = pd.concat([lgb1,ets1])
    BASE = pd.read_pickle(f"721MinTOrder_{method[0]}_L{i}.pkl")[columns]
    BU   = pd.read_pickle(f"721MinTOrder_{method[1]}_L{i}.pkl")[columns]
    TDFP = pd.read_pickle(f"721MinTOrder_{method[2]}_L{i}.pkl")[columns]
    MinT = pd.read_pickle(f"721MinTOrder_{method[3]}_L{i}.pkl")[columns]#_L1
    #lp2_ = {"l10":(0, 3049), "l11":(3049, 3049+4), "l12":( 3049*4, 3049*(14))}
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_BASE  = asl_chcb(lp=lp, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'Shrinkage')
    
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_BU    = asl_chcb(lp=lp, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'OLS')
    
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_TDFP  = asl_chcb(lp=lp, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'Structure', lvls_ = 3, lvls_list_ = [10, 11, 12])#, lvls_ = 3, lvls_list_ = [10, 11, 12]
    
    names     = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m   = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    L_MinT  = asl_chcb(lp=lp, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'Variance', lvls_ = 3, lvls_list_ = [10, 11, 12])
    all_ = pd.concat([L_BASE,L_BU,L_TDFP,L_MinT])
    return all_#mint_ols
for lead_time in [1]:
    print(f"Lead_time is {lead_time}")
    all_ = collect(i= lead_time)
    all_.to_csv(f"721MinTs_IR_L{lead_time}.csv", index = False)

Lead_time is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.65s/it]


## ALL：

In [37]:

def collect(i: int, method):
    lp = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}
    
    columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
    col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 
                 'ets_base', 'ets_bu','ets_td', 'ets_mint']
    names_m   = ['LGBM','LGBM_BU','LGBM_TD','LGBM_MinT',
                 'ETS','ETS_BU','ETS_TD','ETS_MinT']
    lgb1 = pd.read_pickle(f"721lgbInvtSim_L{i}.pkl")[columns] #_L_
    ets1 = pd.read_pickle(f"721etsInvtSim_L{i}.pkl")[columns] #_L_zhme
    BASE = pd.concat([lgb1,ets1])
    BU   = pd.read_pickle(f"721BUOrder_L{i}.pkl")[columns]
    TDFP = pd.read_pickle(f"721TDFPOrder_L{i}.pkl")[columns]
    MinT = pd.read_pickle(f"721MinTOrder_{method}_L{i}.pkl")[columns]#_L1
    #lp2_ = {"l10":(0, 3049), "l11":(3049, 3049+4), "l12":( 3049*4, 3049*(14))}
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint',  
                 'ets_base', 'ets_bu' ,'ets_td', 'ets_mint']
    names_m   = ['LGBM', 'LGBM_BU','LGBM_TD','LGBM_MinT',
                 'ETS',  'ETS_BU', 'ETS_TD', 'ETS_MinT']
    L_BASE  = asl_chcb(lp=lp, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BASE')
    
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint',  
                 'ets_base', 'ets_bu' ,'ets_td', 'ets_mint']
    names_m   = ['LGBM', 'LGBM_BU','LGBM_TD','LGBM_MinT',
                 'ETS',  'ETS_BU', 'ETS_TD', 'ETS_MinT']
    L_BU    = asl_chcb(lp=lp, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BU_IR')
    
    names   = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint',  
                 'ets_base', 'ets_bu' ,'ets_td', 'ets_mint']
    names_m = ['LGBM', 'LGBM_BU','LGBM_TD','LGBM_MinT',
                 'ETS',  'ETS_BU', 'ETS_TD', 'ETS_MinT']
    L_TDFP  = asl_chcb(lp=lp, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TD_IR', lvls_ = 3, lvls_list_ = [10, 11, 12])#, lvls_ = 3, lvls_list_ = [10, 11, 12]
    
    names     = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint',  
                 'ets_base', 'ets_bu' ,'ets_td', 'ets_mint']
    names_m   = ['LGBM', 'LGBM_BU','LGBM_TD','LGBM_MinT',
                 'ETS',  'ETS_BU', 'ETS_TD', 'ETS_MinT']
    L_MinT  = asl_chcb(lp=lp, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'MinT_IR', lvls_ = 3, lvls_list_ = [10, 11, 12])
    all_ = pd.concat([L_BASE,L_BU,L_TDFP,L_MinT])
    return all_#mint_ols
for lead_time in [1]:
    print(f"Lead_time is {lead_time}")
    method_ = "wls-var"
    all_ = collect(i= lead_time, method = method_)
    all_.to_csv(f"8_721Base2IR_L{lead_time}_{method_}.csv", index = False)

Lead_time is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:13<00:00,  1.67s/it]


## draft:

In [34]:
lgb1 = pd.read_pickle(f"721lgbInvtSim_L{1}.pkl")#[columns] #_L_
ets1 = pd.read_pickle(f"721etsInvtSim_L{1}.pkl")#[columns] #_L_zhme
BASE = pd.concat([lgb1,ets1])
BU   = pd.read_pickle(f"721BUOrder_L{1}.pkl")

In [36]:
BASE['name'].unique()

array(['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu',
       'ets_td', 'ets_mint'], dtype=object)

In [33]:
9561664/8

1195208.0

In [27]:
for lead_time in [1,3,7]:
    print(f"Lead_time is {lead_time}")
    method_ = "wls-structure"
    all_ = collect(i= lead_time, method = method_)
    all_.to_csv(f"Base2IR_L{lead_time}_{method_}.csv", index = False)
for lead_time in [1,3,7]:
    print(f"Lead_time is {lead_time}")
    method_ = "ols"
    all_ = collect(i= lead_time, method = method_)
    all_.to_csv(f"Base2IR_L{lead_time}_{method_}.csv", index = False)

Lead_time is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.57s/it]


Lead_time is 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.69s/it]


Lead_time is 7


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.65s/it]


Lead_time is 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.78s/it]


Lead_time is 7


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.62s/it]


In [61]:
columns = ["name","true_demand","backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
lgb1 = pd.read_pickle(f"lgbInvtSim_L{1}.pkl")[columns] #_L_
ets1 = pd.read_pickle(f"etsInvtSim_L{1}.pkl")[columns] #_L_zhme
BASE = pd.concat([lgb1,ets1])
method = "wls-var"
col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
names     = ['lgb_base',  'ets_base']
names_m   = ['LGBM','ETS']
L_BASE  = asl_chcb(lp=lp, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BASE')
L_BASE[L_BASE['Levels'].isin([10,11,12])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.39s/it]


,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs
8,10,LGBM,90%,BASE,0.873483,6.512187,3.861173
10,11,LGBM,90%,BASE,0.885083,3.548454,2.193455
11,12,LGBM,90%,BASE,0.899572,1.953727,1.177339
8,10,LGBM,95%,BASE,0.913859,8.119452,5.253860
10,11,LGBM,95%,BASE,0.920157,4.393018,3.132948
11,12,LGBM,95%,BASE,0.927086,2.382650,1.787983
8,10,LGBM,99%,BASE,0.959050,11.272558,12.375461
10,11,LGBM,99%,BASE,0.959085,6.045235,8.193110
11,12,LGBM,99%,BASE,0.959015,3.215921,5.233622
8,10,ETS,90%,BASE,0.864557,7.173758,4.402604


## test:

In [62]:
TDFP = pd.read_pickle(f"TDFPOrder_L{1}_10-12.pkl")#[columns]
columns = ["name","true_demand","backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
method = "wls-var"
MinT = pd.read_pickle(f"MinTOrder_L3_{method}.pkl")[columns]#_L1
col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
names     = ['lgb_base','lgb_td', 'ets_base','ets_td']
names_m   = ['LGBM','LGBM_TD','ETS','ETS_TD']
lp2 = {"l10":(0, 3049), "l11":(3049, 3049+4), "l12":( 3049*4, 3049*(14))}
td = asl_chcb(lp=lp2, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TD_IR', lvls_ = 3, lvls_list_ = [10, 11, 12])#, lvls_ = 3, lvls_list_ = [10, 11, 12]
td[td['Forecast_Methods'].isin(['LGBM','ETS'])]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.92it/s]


,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs
0,10,LGBM,90%,TD_IR,0.873483,6.512187,3.861173
1,11,LGBM,90%,TD_IR,0.758929,3.969646,4.092265
2,12,LGBM,90%,TD_IR,0.700692,1.060854,4.072834
0,10,LGBM,95%,TD_IR,0.913859,8.119452,5.253860
1,11,LGBM,95%,TD_IR,0.794643,4.982460,7.016086
2,12,LGBM,95%,TD_IR,0.724959,1.188562,7.681112
0,10,LGBM,99%,TD_IR,0.959050,11.272558,12.375461
1,11,LGBM,99%,TD_IR,0.812500,6.997425,29.997284
2,12,LGBM,99%,TD_IR,0.766354,1.441832,32.380559
0,10,ETS,90%,TD_IR,0.864557,7.173758,4.402604
